In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


##### Step 1: Define Paths and Parameters

In [2]:
train_dir = os.path.join('Dataset', 'images.cv_jzk6llhf18tm3k0kyttxz', 'data', 'train')
test_dir = os.path.join('Dataset', 'images.cv_jzk6llhf18tm3k0kyttxz', 'data', 'test')
val_dir = os.path.join('Dataset', 'images.cv_jzk6llhf18tm3k0kyttxz', 'data', 'val')

In [7]:
# Define image size
img_height, img_width = 224, 224

# Define batch size
batch_size = 32

##### Step 2: Create Data Generators

In [4]:
# For training: apply data augmentation and rescaling.

train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    rotation_range=20,        # Random rotations up to 20 degrees
    zoom_range=0.2,           # Random zoom in/out
    horizontal_flip=True,     # Random horizontal flip
    width_shift_range=0.1,    # Random horizontal shifts
    height_shift_range=0.1    # Random vertical shifts
)

In [5]:
# For validation and test sets: only rescaling.

val_test_datagen = ImageDataGenerator(rescale=1./255)

##### Step 3: Load Images from Directories

In [8]:
# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 6225 images belonging to 11 classes.


In [9]:
# Validation generator
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1092 images belonging to 11 classes.


In [10]:
# Test generator
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3187 images belonging to 11 classes.


##### Step 4: Inspect the Loaded Data

In [11]:
print("Train Class Indices:", train_generator.class_indices)
print("Validation Class Indices:", val_generator.class_indices)
print("Test Class Indices:", test_generator.class_indices)

Train Class Indices: {'animal fish': 0, 'animal fish bass': 1, 'fish sea_food black_sea_sprat': 2, 'fish sea_food gilt_head_bream': 3, 'fish sea_food hourse_mackerel': 4, 'fish sea_food red_mullet': 5, 'fish sea_food red_sea_bream': 6, 'fish sea_food sea_bass': 7, 'fish sea_food shrimp': 8, 'fish sea_food striped_red_mullet': 9, 'fish sea_food trout': 10}
Validation Class Indices: {'animal fish': 0, 'animal fish bass': 1, 'fish sea_food black_sea_sprat': 2, 'fish sea_food gilt_head_bream': 3, 'fish sea_food hourse_mackerel': 4, 'fish sea_food red_mullet': 5, 'fish sea_food red_sea_bream': 6, 'fish sea_food sea_bass': 7, 'fish sea_food shrimp': 8, 'fish sea_food striped_red_mullet': 9, 'fish sea_food trout': 10}
Test Class Indices: {'animal fish': 0, 'animal fish bass': 1, 'fish sea_food black_sea_sprat': 2, 'fish sea_food gilt_head_bream': 3, 'fish sea_food hourse_mackerel': 4, 'fish sea_food red_mullet': 5, 'fish sea_food red_sea_bream': 6, 'fish sea_food sea_bass': 7, 'fish sea_food 